说明：checkpoint由于被覆盖了未保存

数据集划分借鉴羽小濯的https://aistudio.baidu.com/aistudio/projectdetail/2145019?channelType=0&channel=0的工程

In [ ]:
#解压一下略小改之后的PaddleSeg，解压一次就可以注释掉了
!unzip -oq /home/aistudio/PaddleSeg.zip

In [ ]:
!unzip -qo data/data100087/B榜测试数据集.zip -d data/


In [ ]:
#解压数据集至data/目录
!unzip -qo data/data95249/train_50k_mask.zip -d data/
!unzip -oq data/data95249/train_image.zip -d data/

# 数据集划分
执行一次就行了，之后可直接跳到后面的参数配置及训练

借鉴羽小濯的https://aistudio.baidu.com/aistudio/projectdetail/2145019?channelType=0&channel=0的工程，在此感谢

In [ ]:
import sys
sys.path.append("PaddleSeg")
import paddleseg
import paddle
import numpy as np
import os
import matplotlib.pyplot as plt
from PIL import Image
from tqdm import tqdm
import random
#设置随机数种子
random.seed(2021)

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/__init__.py:107: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import MutableMapping
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/rcsetup.py:20: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Iterable, Mapping
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/colors.py:53: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Sized


In [ ]:
def write_txt(file_name, imgs_path, labels_path=None, mode='train', val_pro=0.2):
    assert mode=="train" or mode=="test", "ERROR:mode must be train or test."
    if mode!="test":
        train_path = []
        for idx, f_path in enumerate(imgs_path):
            for i_path in sorted(os.listdir(f_path)):
                path1 = os.path.join(f_path, i_path) 
                path2 = os.path.join(labels_path[idx], i_path)
                train_path.append((path1, path2, str(idx)))
        
        if val_pro>=0 and val_pro<=1:
            #打乱数据
            random.shuffle(train_path)
            val_len = int(len(train_path)*val_pro)
            val_path = train_path[:val_len]
            train_path = train_path[val_len:]
            with open(file_name[0], 'w') as f:
                for path in train_path:
                    f.write(path[0]+" "+path[1]+" "+path[2]+"\n")
            with open(file_name[1], 'w') as f:
                for path in val_path:
                    f.write(path[0]+" "+path[1]+" "+path[2]+"\n")  
            return len(train_path), val_len
        else:
            with open(file_name[0], 'w') as f:
                for path in train_path:
                    f.write(path[0]+" "+path[1]+" "+path[2]+"\n") 
            return len(train_path), 0
    else:
        with open(file_name, 'w') as f:
            for path in imgs_path:
                img_path = os.path.join(test_path, path)
                f.write(img_path+"\n")

In [ ]:
def create_txt(data_root, train_imgs_dir=None, train_labels_dir=None, test_dir=None, val_pro=0.2):
    if train_imgs_dir is not None:
        if os.path.exists("train.txt"):
            os.remove("train.txt")
        if os.path.exists("val.txt"):
            os.remove("val.txt")
        train_imgs_dir = os.path.join(data_root, train_imgs_dir)
        train_labels_dir = os.path.join(data_root, train_labels_dir)
        file_names = os.listdir(train_imgs_dir)
        file_names = sorted(file_names)
        train_imgs_path, train_labels_path =[], []
        for na in file_names:
            train_imgs_path.append(os.path.join(train_imgs_dir, na))
            train_labels_path.append(os.path.join(train_labels_dir, na))
        train_len, val_len = write_txt(["train.txt", "val.txt"], train_imgs_path, train_labels_path, mode='train', val_pro=val_pro)
        
        print("训练数据整理完毕！训练集长度：{}，验证集长度：{}， 类别数：{}".format(train_len, val_len, len(file_names)))

    if test_dir is not None:
        if os.path.exists("test.txt"):
            os.remove("test.txt")
        global test_path
        test_path = os.path.join(data_root, test_dir)
        test_imgs_path_list = sorted(os.listdir(test_path))
        write_txt("test.txt", test_imgs_path_list, mode="test")
        print("测试数据整理完毕！测试集长度：{}".format(len(test_imgs_path_list)))

In [ ]:
data_root = "data"
train_imgs_dir = "train_image"
train_labels_dir = "train_50k_mask"
test_dir = "test_image"
create_txt(data_root, train_imgs_dir, train_labels_dir, test_dir, val_pro=0.2)

训练数据整理完毕！训练集长度：40000，验证集长度：10000， 类别数：500
测试数据整理完毕！测试集长度：10989


# 参数配置及训练
本人使用了paddleseg套件，在此像其表示感谢。

因为有paddle推出的高精度语义分割模型ocrnet，所以本人比赛中使用了此模型

·在ocrnet.yml中修改参数配置，包括数据增广使用了paddleseg提供的transformer工具里的    

	- type: RandomHorizontalFlip
    
    - type: RandomVerticalFlip
    
    - type: RandomBlur       提高0.4%个精确度
    
    - type: ResizeStepScaling  
    
    - type: RandomDistort    提高0.4%个精确度
    
·输入图像大小被resize到【256,256】是因为图像太大运行速度特别慢，对提升精度也没有特别好的帮助

·PolynomialDecay作为学习率衰减方式，0.03的学习率损失下降较快

·使用Diceloss作为损失函数相比交叉熵损失更加接近IOU的评价标准，提高2%个精确度


In [14]:
!python PaddleSeg/train.py --config ocrnet.yml --do_eval --use_vdl \
--save_dir /home/aistudio/output_ocrnet --save_interval 2000 \
                    # --resume_model /home/aistudio/output_ocrnet/iter_26000 


2021-08-09 15:07:19 [INFO]	
------------Environment Information-------------
platform: Linux-4.4.0-150-generic-x86_64-with-debian-stretch-sid
Python: 3.7.4 (default, Aug 13 2019, 20:35:49) [GCC 7.3.0]
Paddle compiled with cuda: True
NVCC: Cuda compilation tools, release 10.1, V10.1.243
cudnn: 7.6
GPUs used: 1
CUDA_VISIBLE_DEVICES: None
GPU: ['GPU 0: Tesla V100-SXM2-32GB']
GCC: gcc (Ubuntu 7.5.0-3ubuntu1~16.04) 7.5.0
PaddlePaddle: 2.1.0
OpenCV: 4.1.1
------------------------------------------------
2021-08-09 15:07:19 [INFO]	
---------------Config Information---------------
batch_size: 40
iters: 30000
loss:
  coef:
  - 1
  - 0.4
  types:
  - ignore_index: 255
    type: DiceLoss
lr_scheduler:
  end_lr: 0
  learning_rate: 0.03
  power: 0.9
  type: PolynomialDecay
model:
  backbone:
    align_corners: false
    pretrained: https://bj.bcebos.com/paddleseg/dygraph/hrnet_w48_ssld.tar.gz
    type: HRNet_W48
  backbone_indices:
  - -1
  pretrained: null
  type: OCRNet
optimizer:
  momentum: 0.9

# 推理
已在PaddleSeg中做了修改可以直接预测出结果
最后在推理时使用数据增广可以提高预测精度，但是本人由于没来及提交更好的结果并没有体现到比赛成绩上，B榜分数为0.76425

In [ ]:
#推理
!python PaddleSeg/predict.py --config ocrnet.yml \
# --model_path output_ocrnet/best_model/model.pdparams \
--model_path model.pdparams\    #该模型为较好的最终训练结果，checkpoint由于被覆盖了未保存
--image_path data/test_image --save_dir output/result_1 --aug_pred --flip_horizontal --flip_vertical

# 压缩结果，提交文件
[第三届中国AI+创新创业大赛：半监督学习目标定位竞赛](https://aistudio.baidu.com/aistudio/competition/detail/78)


In [ ]:
%cd output/result_1/results
!zip -r -oq /home/aistudio/pred.zip ./
%cd /home/aistudio